In [1]:
#https://stackoverflow.com/questions/18130499/how-to-scrape-instagram-with-beautifulsoup
#for maco sx, download chrome driver - https://sites.google.com/a/chromium.org/chromedriver/downloads

In [3]:
import re
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import csv

In [4]:
def readCSV (file):
    df = pd.read_csv(file)
    return df


In [5]:


#URL = 'https://www.instagram.com/explore/tags/chapelhill/?hl=en' #hoboken location

def instaURL (URL):
    driver = webdriver.Chrome('/Users/Matt/Downloads/chromedriver')  # Optional argument, if not specified will search path.
    driver.get(URL);
    time.sleep(1) # Let the user actually see something!
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Click on "Load more..." label
    load_more = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div[3]/a')
    load_more.click()
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source)
    time.sleep(5) # Let the user actually see something!
    driver.quit()
    
    #isolate the owner IDs from the pictures
    stringTest=""
    for x in soup:
        stringTest = stringTest + str(x)
        #print x
    pic_pattern='''(?<="owner").*?(?="owner")'''
    pic = re.findall(pic_pattern, stringTest)
    output = pic
    #print output
    return output


In [6]:
def formatHashtag(hashtag):
    
    outputTag=""
    hashtagDict = []
    #print hashtag
    listRange = range(len(hashtag))
    for i in listRange:
        iterator = i
        for x in hashtag[(i+1):]:
            outputTag+=hashtag[i]+ "," + (x) + "\n"
    #print outputTag
    return outputTag

In [7]:
def parse(Source, City):
    hashtagout=""
    df = pd.DataFrame()
    CityFileName = str('InstaOutputMain_'+City+'.csv')
    CityFileListName = str('InstaOutputlist_'+City+'.csv')
    ofile  = open(CityFileName, "wb")
    ofile.write("date" + "," + "pictureURL" +","+ "numLikes" +","+ "City" + "," + "hashtagout" + "\n")
    #ofile_list  = open(CityFileListName, "wb") create
    ofile_list  = open(CityFileListName, "a+") #append
    ofile_list.write("tag1" + "," + "tag2"+ "\n")
    for test in Source:  
        ownerID_pattern='''(?<=: {"id": ").*?(?="},)'''
        ownerID = re.search(ownerID_pattern, test)
        ownerID = ownerID.group(0)
        #print ownerID

        video_pattern='''(?<="is_video": ).*?(?=,)'''
        video = re.search(video_pattern, test)
        if video: video = video.group(0)
        else: video="N/A"

        pictureURL_pattern='''(?<="display_src": ").*?(?="},)'''
        pictureURL = re.search(pictureURL_pattern, test)
        if pictureURL: pictureURL = pictureURL.group(0)
        else: pictureURL="N/A"

        date_pattern='''(?<="date": ).*?(?=,)'''
        date = re.search(date_pattern, test)
        date = date.group(0)
        date = datetime.datetime.fromtimestamp(int(date)).strftime('%Y-%m-%d %H:%M:%S')

        commentsCount_pattern='''(?<="comments": {"count": ).*?(?=},)'''
        commentsCount = re.search(commentsCount_pattern, test)
        numComments = commentsCount.group(0)

        numLikes="N/A"
        likesCount_pattern='''(?<="likes": {"count": ).*?(?=},)'''
        likesCount = re.search(likesCount_pattern, test)
        numLikes = likesCount.group(0)

        caption_pattern='''(?<="caption": ").*?(?=, "likes":)'''
        caption = re.search(caption_pattern, test)
        hashtagtext="N/A"
        if caption:
            hashtagtext = caption.group(0)

        hashtag = "N/A"
        hashtag_pattern='''(?<=#).*?(?=\W)'''
        hashtag = re.findall(hashtag_pattern, hashtagtext)

        
        #print hashtag
        df = df.append({ 'ownerID': ownerID,
                           'isVideo': video,
                           'pictureURL': pictureURL,
                           'date': date,
                           'numComments': numComments,
                           'numLikes': numLikes,
                           'caption': hashtagtext,
                        'hashtag': hashtag
            }, ignore_index=True)
        #additional data cleanup
        cols = df.columns.tolist()
        cols = ['caption',
         'date',
         'isVideo',
         'numComments',
         'numLikes', 'pictureURL', 'ownerID','hashtag']
        df=df[cols]

        #print hashtag
        
        
        formatHashtag(hashtag)
        ofile_list.write(formatHashtag(hashtag))

        hashtagout=""
        for x in hashtag:
            hashtagout += x + "\t"
            
        
        ofile.write(date + "\t" + pictureURL +"\t"+ numLikes +"\t"+ City + "\t" + hashtagout + "\n")

    return "Check InstaOutput.txt file for output"
    

In [8]:
def getCities(df):
    for index, row in df.iterrows():
        URL= row['Link']
        City = row['City']
        output=instaURL(URL)
        parse(output,City)


In [9]:
file = '/Users/Matt/Documents/Stevens/BIA 658 Social Network Analytics/Instagram/Data/Cities List.csv'
df = readCSV(file)
#df['Link'][1]
getCities(df)

/Users/Matt/anaconda2/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [52]:
#output=instaURL(URL)


In [53]:
#parse(output)